In [1]:
#!pip install beautifulsoup4
#!pip install requests
#!conda install pandas
#!pip install tqdm

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [8]:
def clean_text(text):
    return text.replace('/', '').strip().lower()


def process_mode(row):
    return clean_text(row.td.text)


def process_times(row):
    return tuple(clean_text(col.text) for col in row.find_all('td'))


def process_conj(row, mode, times, infinitive):
    verbs = []
    cols = row.find_all('td')
    
    for i in range(0, 6, 2):
        verbs.append(
            dict(infinitive=infinitive,
                 pess=cols[i].text,
                 conj=cols[i + 1].text,
                 time=times[i // 2],
                 mode=mode
                )
        )
        
    return verbs



def get_indicativo(infinitive:str, table:list) -> list:
    
    verbs = []
    
    for i, row in enumerate(table.find_all('tr')):
        if i == 3:
            mode = process_mode(row)
        if i == 4 or i == 12:
            times = process_times(row)
        if 5 <= i <= 10 or 14 <= i <= 19:
            verbs.extend(process_conj(row, mode, times, infinitive))
    
    return verbs

def get_conjugations(verb):
    r = requests.get(f"http://www.conjuga-me.net/verbo-{verb}")
    soup = BeautifulSoup(r.text, 'html.parser')
    indicativo = get_indicativo(verb, soup.find(class_="conj"))
    return indicativo

In [9]:
def clean_verb(verb):
    return verb.replace('[', '').replace(']', '').lower().strip()

#verbs = pd.read_clipboard().assign(verb=lambda df: df.verb.apply(clean_verb)).verb.values
verbs = pd.read_csv('top_1000_verbs_pt.csv').verbs.values
verbs[:1]

array(['ser'], dtype=object)

In [10]:
parsed_verbs = verbs[:1]

In [13]:
parsed = []

for verb in tqdm(verbs[:100]):
    parsed.extend(get_conjugations(verb))


100%|██████████| 100/100 [01:38<00:00,  1.17s/it]

In [14]:
pd.DataFrame(parsed).to_csv('top_100_pt.csv', index=False)

In [13]:
verbs[:100]

array(['ser', 'ter', 'estar', 'poder', 'fazer', 'ir', 'haver', 'dizer',
       'dar', 'ver', 'saber', 'querer', 'ficar', 'dever', 'passar', 'vir',
       'chegar', 'falar', 'deixar', 'encontrar', 'levar', 'começar',
       'partir', 'pensar', 'parecer', 'apresentar', 'olhar', 'tornar',
       'sair', 'voltar', 'conseguir', 'achar', 'existir', 'sentir',
       'entrar', 'chamar', 'conhecer', 'considerar', 'pôr', 'continuar',
       'viver', 'ouvir', 'tomar', 'acabar', 'receber', 'perder', 'andar',
       'trabalhar', 'criar', 'pedir', 'seguir', 'contar', 'acontecer',
       'afirmar', 'tratar', 'esperar', 'gostar', 'usar', 'manter',
       'realizar', 'abrir', 'escrever', 'permitir', 'ocorrer', 'mostrar',
       'lembrar', 'trazer', 'procurar', 'morrer', 'tentar', 'formar',
       'aparecer', 'incluir', 'cair', 'correr', 'ganhar', 'surgir',
       'nascer', 'pagar', 'representar', 'entender', 'produzir', 'ler',
       'precisar', 'perguntar', 'constituir', 'colocar', 'possuir',
       '

In [14]:
decks = [
    list(verbs[:10]),
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
]

In [18]:
pd.DataFrame(verbs).rename(columns={0:'verbs'}).to_csv('top_1000_verbs_pt.csv', index=False)